## Camping Alone

You have your camping site in the woods. This camping has last for a very long time. You want to go back home, but you got trapped here. You know how to survive here by yourself.

In [9]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

In [10]:
camping_site = things.Location(
    "Camping Site",
    "You are at your little safe camping site."
)

woods = things.Location(
    "Woods",
    "You are standing in the middle of the woods. Trees and greens are around you.",
)

river = things.Location(
    "River",
    "You are standing in front of a beautiful, clean river. You can see fish and crabs in it."
)

camping_site.add_connection("west", woods)
woods.add_connection("west", river)

In [11]:
branch = things.Item(
    "branch",
    "a tree branch",
    "A THICK BRANCH YOU FOUND IN THE WOODS.",
)
woods.add_item(branch)

bow_and_arrow = things.Item(
    "bow and arrow",
    "a nice bow and some sharp arrows",
    "YOU MADE BOW AND ARROW FROM THE BRANCH!",
)

rabbit = things.Item(
    "rabbit",
    "a white rabbit",
    "A WHITE RABBIT YOU GOT FROM THE WOODS.",
)
woods.add_item(rabbit)
rabbit.set_property("trapped", False)
rabbit.set_property("gettable", False)

snake = things.Item(
    "snake",
    "a poisonous snake",
    "THIS POISONOUS SNAKE IS VERY DANGEROUS.",
)
woods.add_item(snake)
snake.set_property("trapped", False)
snake.set_property("gettable", False)

fish = things.Item(
    "fish",
    "a black fish",
    "A FISH YOU GOT FROM THE RIVER.",
)
river.add_item(fish)
fish.set_property("trapped", False)
fish.set_property("gettable", False)

torch = things.Item(
    "torch",
    "a torch",
    "A TORCH YOU PLACED AT YOUR CAMPING SITE.",
)
camping_site.add_item(torch)
torch.set_property("is_lightable", True)
torch.set_property("is_lit", False)
torch.add_command_hint("light torch!")

apple = things.Item(
    "apple",
    "an red apple",
    "AN RED APPLE THAT FELL DOWN FROM A TREE.",
)
woods.add_item(apple)

In [12]:
player = things.Character(
    name="The player",
    description="You are trapped here. You are trying to survive.",
    persona="I feel lonely and hungry and cold.",
)

In [13]:

class Camping(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        if fish.get_property("trapped") and rabbit.get_property("trapped") and torch.get_property("is_lit") and player.is_in_inventory(apple):
            self.parser.ok("You survived today. You win!")
            return True
        elif snake.get_property("trapped"):
            self.parser.ok("You got too close to the snake and got bitten. You lose!")
            return True
        else:
            return False


In [14]:
class Craft(actions.Action):
    ACTION_NAME = 'craft'
    ACTION_DESCRIPTION = 'Craft useful tools'

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)

    def check_preconditions(self) -> bool:
        if not self.character.is_in_inventory(branch):
            self.parser.fail("You don't have the material. Find some!")
            return False
        if self.character.is_in_inventory(bow_and_arrow):
            self.parser.fail("You already crafted the tool!")
            return False
        return True

    def apply_effects(self):
        self.character.remove_from_inventory(branch)
        self.character.add_to_inventory(bow_and_arrow)
        self.parser.ok("You crafted bow and arrows. Now you are able to hunt!")

In [15]:
class Hunt(actions.Action):
    ACTION_NAME = 'hunt'
    ACTION_DESCRIPTION = 'Hunt and find some animals'

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)

    def check_preconditions(self) -> bool:
        print(self.character.location.name)
        if self.character.location.name == "Camping Site":
            self.parser.fail("There is no animal near your camping site.")
            print("1")
            return False
        print("2")
        return True

    def apply_effects(self):
        if self.character.location.name == "River":
            print("3")
            if self.character.is_in_inventory(bow_and_arrow):
                self.parser.ok("You got a fish!")
                fish.set_property("trapped", True)
            else:
                self.parse.fail("You need tools for hunting!")
        elif self.character.location.name == "Woods":
            print("4")
            if self.character.is_in_inventory(bow_and_arrow):
                self.parser.ok("You got a rabbit!")
                rabbit.set_property("trapped", True)
            else:
                self.parser.ok("You got too close to the snake!")
                snake.set_property("trapped", True)
        print("5")


In [16]:
game = Camping(camping_site, player, custom_actions=[Craft, Hunt])

game.game_loop()

You are at your little safe camping site.
Exits:
West to Woods

You see:
 * a torch
        light torch!


The player got the torch.
You are standing in the middle of the woods. Trees and greens are around you.
Exits:
East to Camping Site
West to River

You see:
 * a tree branch
 * a white rabbit
 * a poisonous snake
 * an red apple


The player got the apple.
I'm not sure what you want to do.
The player got the branch.
You crafted bow and arrows. Now you are able to hunt!
Woods
2
4
You got a rabbit!
5
You are standing in front of a beautiful, clean river. You can see fish and
crabs in it.
Exits:
East to Woods

You see:
 * a black fish


River
2
3
You got a fish!
5
The player lights the torch. It glows.
You survived today. You win!
